import some librares

In [2]:
import time
from collections import namedtuple


import sys

import numpy as np

import keras as ks
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU, Dropout, BatchNormalization
from keras.layers import Input, Dense, Embedding
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import random




Using TensorFlow backend.


In [132]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving My.txt to My.txt
User uploaded file "My.txt" with length 522237 bytes


read source text (from books.txt)..

In [0]:
with open('My.txt', 'r') as f:
    text=f.read()
chars = sorted(list(set(text)))


char2id = dict((c, i) for i, c in enumerate(chars))

In [0]:
seq_size = 100

prepare the dataset of input to output pairs encoded as integers

In [0]:
def get_batches(batch_count=128, batch_size=64):
    batch_x = np.zeros((batch_size, seq_size))
    batch_y = np.zeros((batch_size, len(chars)))
    for bi in range(batch_count):
        for seq_index in range(batch_size):
            pos = random.randint(0, len(text) - seq_size - 10)
            sequence = text[pos:pos+seq_size]
            next_char = text[pos+seq_size]
            for i in range(seq_size):
                batch_x[seq_index,i] = char2id[sequence[i]]
            batch_y[seq_index, :] = 0
            batch_y[seq_index, char2id[next_char]] = 1
        yield batch_x, batch_y

try to build model..

In [0]:
model = Sequential()
model.add(Embedding(len(chars), 32))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

fit the model

In [0]:
def sample_character(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
for epoch in range(30):
    print("epoch: {}".format(epoch))
    losses = []
    for batch_x, batch_y in get_batches():
        loss = model.train_on_batch(batch_x, batch_y)
        losses.append(loss)
    print("train_loss: {}".format(np.mean(losses)))
    
    if epoch > 25 :
        max_gen_count = 500
        index = random.randint(0, len(text) - 100)
        for t in [0.1, 0.4, 0.7, 1]:
            print("t={}".format(t))

            sentence = text[index: index + seq_size]
            generated = sentence

            for step in range(max_gen_count):
                batch = np.zeros((1, seq_size,))
                for i in range(seq_size):
                    batch[0,i] = char2id[sentence[i]]
                id = sample_character(model.predict_on_batch(batch)[0], t)
                generated += chars[id]
                sentence = sentence[1:] + chars[id]
            print(generated)